In [2]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy import spatial
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from hdx.data.resource import Resource
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
import nltk
import re 
import gensim
import os;
import re 
import pandas as pd
import re;
import logging;
import sqlite3;
import time;
import sys;
import multiprocessing;
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
from io import StringIO
from sklearn.feature_extraction.text import CountVectorizer
import re
  
import pandas as pd
import requests

In [3]:
import warnings


warnings.filterwarnings('ignore')

In [4]:
#import import_ipynb
import  Utility

## Keyword Extraction (by Topic extraction),  Tag expansion, distance computing, and scoring similar tags for the datasets.
<ol>
<li> Read the preprocessed content </li>
<li> Generate a set of topics for the collection of the datasets</li>
<li> Extract top words for each topic </li>
<li> Expand tags by word2vec model </li>
<li> Find the topic for each datset </li>
<li> Represent the top words of the dataset's topic by word2vec model </li>
<li> Represent HDX tags by word2vec model </li>
<li> Compute the similarity between the HDX tags vs dataset </li>
</ol>




In [5]:
# concatenate the content of all metadata fields into raw text
def get_all_text(df):
    #df.tags.astype(str)+ " " +
    textdata =  df.title.fillna('').astype(str)+" " + df.header.fillna('').astype(str) +" " + df.organization.fillna('').astype(str) +" " +  df.notes.fillna('').astype(str)+" "+ df.country.fillna('').astype(str) + " " + df.geodata.fillna('').astype(str) 
    return textdata

In [6]:
df=pd.read_csv("after_clean_v3.csv")#,delim_whitespace = True)



In [7]:
df['header'] = Utility.data_clean(df,['header'])

In [8]:
stoplist= set(stopwords.words('english'))
stoplist.update(['type','etc','unnamed','nan','file','xls','xlsx','etc','zip','link',
                 'description','https','name'])

In [9]:
df['All_text'] = get_all_text(df)

In [10]:
df['text']=  Utility.remove_stopwords(df['All_text'],stoplist)

In [11]:
evaluation_set = pd.read_csv('good_test.csv')

In [12]:
## convert the string of list (tags) to list of (tags) #problem for reading list from CSV
from ast import literal_eval
evaluation_set.loc[:,'perfect_tags'] = evaluation_set.loc[:,'perfect_tags'].apply(lambda x: literal_eval(x))

In [13]:
# tag list defined by HDX team
tag_list_clean = Utility.read_concept_tags()
len(tag_list_clean)

364

In [14]:
# Google pretrained word2vec model:
model, word = Utility.get_model()
index2word_set = set(model.wv.index2word)

In [15]:
##Tag expansion - Optional
new_tag_list_clean = []
for tag in tag_list_clean:
    expanded = Utility.expan_tags(tag, model , 5)
    if expanded =="":
        new_tag_list_clean.append(tag)
    else:
        new_tag_list_clean.append(expanded+" "+tag)
    

In [16]:
##final list
## represent the tfidf of the documents into word2vec
##recommend tags from the clean_list to each dataset using the 
for i, row in evaluation_set.iterrows():
    doc = evaluation_set.ix[i]['tfidf']
    suggested_tags =Utility.cosine_distance_list(doc, tag_list_clean,model,index2word_set,evaluation_set.ix[i]['k']+10)
    #print(suggested_tags)
    evaluation_set.at[i, 'rec'] = suggested_tags

In [17]:
pred_list= evaluation_set['rec']

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=True,use_idf=True,
                               max_df=.9, min_df=.1,#0.2 0.9 .78 ,.18
                                #max_features=100, #ngram_range =(1,2),#analyzer ='word',
                                #ngram_range =(1,2),
                                #analyzer ='word'
                               stop_words=stoplist
                            )
Tfidf = vectorizer.fit_transform(df['All_text'].values.astype('U'))

In [19]:
# Define Search Param using GridSearch
#vectorizer
from sklearn.model_selection import GridSearchCV
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
LDA_model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
LDA_model.fit(Tfidf)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
# Best Model
best_lda_model = LDA_model.best_estimator_
# Model Parameters
print("Best Model's Params: ", LDA_model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", LDA_model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(Tfidf))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -3005.390721738919
Model Perplexity:  65.90043712932051


In [21]:
## use the saved the model directly without training.
import pickle
ldafile = open('lda_model_sym_wiki.pkl','wb')
pickle.dump(lda,ldafile)
ldafile.close()

In [22]:
# print the top words for each topic
print("\nTopics in LDA model:")
tf_feature_names = vectorizer.get_feature_names()
topics_words =Utility.print_top_words(best_lda_model, tf_feature_names, 8)
topics_words =Utility.print_topics(best_lda_model, vectorizer, 8)


Topics in LDA model:
Topic 0: indicators socioeconomic sustainable data development demographic education covering
Topic 1: data education sustainable indicators socioeconomic various contains demographic
Topic 2: population education compiled collected data world produced dataset
Topic 3: socioeconomic sustainable indicators demographic development unesco data education
Topic 4: education statistics data indicators socioeconomic sustainable health contains
Topic 5: urban statistics indicators national global surveys local observatory
Topic 6: dataset education compiled social team indicators contains socioeconomic
Topic 7: indicators socioeconomic sustainable development data demographic education various
Topic 8: education dataset data contains indicators national world health
Topic 9: world health global household available data urban education

Topic 0:
[('indicators', 0.6823884528023729), ('socioeconomic', 0.6590402479808948), ('sustainable', 0.6541891737029326), ('data', 0.58646

In [23]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(Tfidf)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(df['All_text']))]
# createdataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get main topic for each document
main_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['main_topic'] = main_topic
# Styling

# Apply Style for the main topic for each dataset
df_document_topics = df_document_topic.head(15).style.applymap(Utility.color_green).applymap(Utility.make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,main_topic
Doc0,0.02,0.02,0.02,0.02,0.02,0.85,0.02,0.02,0.02,0.02,5
Doc1,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,8
Doc2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,8
Doc3,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,8
Doc4,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.62,0.04,8
Doc5,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc6,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc7,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc8,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.43,0.23,8
Doc9,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.62,0.04,8


In [24]:
# create Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index based vectorizer words
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,additionally,affairs,analyses,analyze,analyzed,authorities,available,censuses,civil,collect,...,surveys,sustainable,team,transport,unesco,united,urban,urbanization,various,world
Topic0,0.136688,0.131738,0.137082,0.127168,0.130020,0.131773,0.133007,0.130779,0.131889,0.147813,...,0.151739,0.654189,0.129025,0.134670,0.394595,0.162000,0.145233,0.131743,0.385710,0.160416
Topic1,0.128988,0.127905,0.137085,0.133525,0.129474,0.133155,0.132492,0.133584,0.131657,0.126787,...,0.126183,0.191987,0.138937,0.129792,0.165657,0.158162,0.129137,0.129564,0.177624,0.132662
Topic2,0.130284,0.127933,0.131322,0.130035,0.133542,0.131451,0.127275,0.128999,0.138597,0.130226,...,0.128288,0.171274,0.138782,0.126908,0.151998,0.142494,0.129036,0.131633,0.151092,1.072839
Topic3,0.126055,0.126018,0.132769,0.129831,0.129924,0.134146,0.129088,0.130689,0.130161,0.130010,...,0.136257,0.173457,0.129196,0.125192,0.154663,0.134392,0.128387,0.126733,0.145866,0.131617
Topic4,0.135848,0.133214,0.128365,0.135803,0.139232,0.138050,0.138396,0.141838,0.136267,0.135022,...,0.145621,0.170838,0.129672,0.135406,0.154617,0.146753,0.146290,0.127995,0.153729,0.137595


In [25]:
# view
df_topic_keywords

,additionally,affairs,analyses,analyze,analyzed,authorities,available,censuses,civil,collect,...,surveys,sustainable,team,transport,unesco,united,urban,urbanization,various,world
Topic0,0.136688,0.131738,0.137082,0.127168,0.130020,0.131773,0.133007,0.130779,0.131889,0.147813,...,0.151739,0.654189,0.129025,0.134670,0.394595,0.162000,0.145233,0.131743,0.385710,0.160416
Topic1,0.128988,0.127905,0.137085,0.133525,0.129474,0.133155,0.132492,0.133584,0.131657,0.126787,...,0.126183,0.191987,0.138937,0.129792,0.165657,0.158162,0.129137,0.129564,0.177624,0.132662
Topic2,0.130284,0.127933,0.131322,0.130035,0.133542,0.131451,0.127275,0.128999,0.138597,0.130226,...,0.128288,0.171274,0.138782,0.126908,0.151998,0.142494,0.129036,0.131633,0.151092,1.072839
Topic3,0.126055,0.126018,0.132769,0.129831,0.129924,0.134146,0.129088,0.130689,0.130161,0.130010,...,0.136257,0.173457,0.129196,0.125192,0.154663,0.134392,0.128387,0.126733,0.145866,0.131617
Topic4,0.135848,0.133214,0.128365,0.135803,0.139232,0.138050,0.138396,0.141838,0.136267,0.135022,...,0.145621,0.170838,0.129672,0.135406,0.154617,0.146753,0.146290,0.127995,0.153729,0.137595
Topic5,13.089947,13.057107,12.897151,12.902426,12.858050,25.659099,13.094181,25.659103,12.888784,13.359408,...,25.972872,0.184918,12.743876,25.739987,0.157375,15.417103,102.503681,12.899305,0.274940,12.251585
Topic6,0.132708,0.129096,0.129115,0.126389,0.130184,0.130364,0.131975,0.129132,0.127358,0.131233,...,0.128042,0.143360,0.158758,0.129396,0.133055,0.128833,0.129080,0.128672,0.130525,0.127481
Topic7,0.133187,0.132389,0.128684,0.127680,0.139530,0.123328,0.151225,0.126205,0.126537,0.126655,...,0.135440,73.227748,0.128159,0.141974,36.725915,2.148359,0.130327,0.130403,37.584138,0.200028
Topic8,0.142096,0.278797,0.131476,0.133850,0.648221,0.133384,1.522266,0.131430,0.133762,0.311047,...,0.126387,0.266769,1.920720,0.129852,0.147457,2.301863,0.131117,0.138909,0.154084,3.105374
Topic9,0.165169,0.278967,0.158522,0.156804,0.156207,0.194781,1.107797,0.187003,0.160819,0.164002,...,0.185866,0.138022,0.159937,0.188949,0.134303,0.157376,0.436489,0.156029,0.142658,39.659266


In [26]:
# 
topic_keywords = Utility.show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,indicators,socioeconomic,sustainable,data,development,demographic,education,covering,unesco,portal,various,contains,dataset,united,world
Topic 1,data,education,sustainable,indicators,socioeconomic,various,contains,demographic,development,unesco,dataset,portal,united,health,covering
Topic 2,population,education,compiled,collected,data,world,produced,dataset,contains,indicators,socioeconomic,sustainable,global,demographic,development
Topic 3,socioeconomic,sustainable,indicators,demographic,development,unesco,data,education,covering,contains,portal,various,surveys,united,authorities
Topic 4,education,statistics,data,indicators,socioeconomic,sustainable,health,contains,demographic,development,unesco,portal,various,covering,united
Topic 5,urban,statistics,indicators,national,global,surveys,local,observatory,transport,censuses,authorities,health,data,economic,demographic
Topic 6,dataset,education,compiled,social,team,indicators,contains,socioeconomic,sustainable,global,demographic,policy,household,produced,covering
Topic 7,indicators,socioeconomic,sustainable,development,data,demographic,education,various,covering,portal,unesco,contains,united,published,dataset
Topic 8,education,dataset,data,contains,indicators,national,world,health,collected,united,team,conducted,local,available,economic
Topic 9,world,health,global,household,available,data,urban,education,statistics,affairs,indicators,national,authorities,local,observatory


In [30]:
txt = [df.ix[100]['All_text']]

In [31]:
txt

['venezuela bolivarian republic sustainable development education demographic socioeconomic indicators venezuela bolivarian republic sustainable development education demographic socioeconomic indicators  contains data from unesco data portal covering various indicators venezuela ']

In [32]:
 evaluation_set.ix[0]['All_text']

'education health hxl population transport education health hxl population transport vanuatu demographic health education transport indicator category indicator indicator friendly type data latitude longitude region id country id name year value urban indicator data available analyzed compiled published un habitat global urban observatory support government local authority civil society organization develop urban indicator data statistic urban statistic collected household survey census conducted national statistic authority global urban observatory team analysis compiles urban indicator statistic survey census additionally local urban observatory collect compile analyze urban data national policy development population statistic produced united nation department economic social affair world urbanization prospect vanuatu'

In [33]:
# Predict the topic
#mytext =
infer_topic, topic, prob_scores = Utility.find_topic(df_topic_keywords = df_topic_keywords, text = txt,vectorizer = vectorizer, best_lda_model=best_lda_model)
print(topic)

#print(infer_topic)

['socioeconomic', 'sustainable', 'development', 'data', 'demographic', 'education', 'various', 'covering', 'portal', 'unesco', 'contains', 'united', 'published']


In [34]:
topic_words = ""
for word in topic: 
        #print(k)
        topic_words+=word +" "

In [35]:
topic_words

'socioeconomic sustainable development data demographic education various covering portal unesco contains united published '

In [45]:
## example of cosine distance based on the top-words of the topic 
Utility.cosine_distance(topic_words, tag_list_clean,model,index2word_set,10) #evaluation_set['LDA'].ix[20]

[('social development centers', 0.58205813),
 ('educational attainment', 0.57239497),
 ('sustainable development', 0.56834835),
 ('social media data', 0.56815886),
 ('education cluster', 0.5346632),
 ('sex and age disaggregated data - sadd', 0.5262456),
 ('access to education', 0.5258083),
 ('demographics', 0.5151829),
 ('sustainable development goals - sdg', 0.49578387),
 ('cultural sites', 0.49301958)]

In [37]:
#evaluation_set['LDA'] =" "
# get the document that we want to extract keywords from
for i, row in evaluation_set.iterrows():
    candidates=""
    doc =[evaluation_set.ix[i]['All_text']]
    #print(doc)
    infer_topic, topic, prob_scores = Utility.find_topic(df_topic_keywords =df_topic_keywords, text = doc ,vectorizer = vectorizer, best_lda_model=best_lda_model)
    #print(topic)
    for k in topic: 
        #print(k)
        candidates+=k +" "
    evaluation_set.at[i,'LDA'] = candidates  
 

In [38]:
evaluation_set.ix[0]['LDA']

'statistics indicators national global surveys local observatory transport censuses authorities health data economic '

In [39]:
actual_list= evaluation_set['tag_list']
pred_list= evaluation_set['rec']
LDA_list = evaluation_set['LDA']
perfect_list = evaluation_set['perfect_tags']

In [40]:
####compare top words by topic modeling with the perfect list:
result_lda=[]
for document_lst, pred_tags in zip(perfect_list,LDA_list):
    #print("actual" , document_lst)
    #print("tfidf: " , pred_tags)
    matching= [s for s in document_lst if any(s in xs or xs in s for xs in [pred_tags])]
    #print("matching", matching)
    result_lda.append((len(matching), len(document_lst)))
### compuate the recall for the keywords 
lda_recall = [] 
for t in result_lda:
    lda_recall.append(t[0]/t[1])
np.average(lda_recall)
        

0.31346801346801345

In [41]:
#good_tags = Utility.remove_stopwords(pd.Series(P_tags))

In [42]:
##final list
## represent the tfidf of the documents into word2vec
##recommend tags from the clean_list to each dataset using the 
for i, row in evaluation_set.iterrows():
    doc = evaluation_set.ix[i]['tfidf']
    suggested_tags =Utility.cosine_distance_list(doc, tag_list_clean,model,index2word_set,evaluation_set.ix[i]['k']+10)
    #print(suggested_tags)
    evaluation_set.at[i, 'rec'] = suggested_tags

In [43]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(best_lda_model, Tfidf, vectorizer)

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.149250 -0.102180       1        1  43.378941
7     -0.316570 -0.017396       2        1  31.857491
8     -0.135658  0.220419       3        1   7.461834
9      0.243131  0.169190       4        1   3.499299
2      0.022866  0.050246       5        1   2.845822
0     -0.071035 -0.061537       6        1   2.191388
4      0.026068 -0.061486       7        1   2.191307
1      0.017052 -0.060520       8        1   2.191307
6      0.036844 -0.061417       9        1   2.191305
3      0.028053 -0.075318      10        1   2.191305, topic_info=   Category        Freq           Term       Total  loglift  logprob
24  Default  123.000000      education  123.000000  30.0000  30.0000
57  Default   60.000000          world   60.000000  29.0000  29.0000
38  Default   34.000000     population   34.000000  28.0000  28.0000
18  Default   43.000000        dataset   43.000000  27.0000  27.0000
45  Default   79.000000  socioeconomic   79.000000  26.0000  26.0000
49  Default   79.000000    sustainable   79.000000  25.0000  25.0000
17  Default   98.000000           data   98.000000  24.0000  24.0000
22  Default   74.000000    development   74.000000  23.0000  23.0000
30  Default  126.000000     indicators  126.000000  22.0000  22.0000
54  Default   94.000000          urban   94.000000  21.0000  21.0000
19  Default   72.000000    demographic   72.000000  20.0000  20.0000
25  Default   33.000000         global   33.000000  19.0000  19.0000
28  Default   35.000000         health   35.000000  18.0000  18.0000
46  Default   67.000000     statistics   67.000000  17.0000  17.0000
56  Default   42.000000        various   42.000000  16.0000  16.0000
52  Default   41.000000         unesco   41.000000  15.0000  15.0000
39  Default   42.000000         portal   42.000000  14.0000  14.0000
16  Default   42.000000       covering   42.000000  13.0000  13.0000
15  Default   47.000000       contains   47.000000  12.0000  12.0000
12  Default   21.000000       compiled   21.000000  11.0000  11.0000
32  Default   38.000000       national   38.000000  10.0000  10.0000
48  Default   26.000000        surveys   26.000000   9.0000   9.0000
10  Default   20.000000      collected   20.000000   8.0000   8.0000
7   Default   26.000000       censuses   26.000000   7.0000   7.0000
5   Default   26.000000    authorities   26.000000   6.0000   6.0000
51  Default   26.000000      transport   26.000000   5.0000   5.0000
35  Default   26.000000    observatory   26.000000   4.0000   4.0000
31  Default   28.000000          local   28.000000   3.0000   3.0000
23  Default   21.000000       economic   21.000000   2.0000   2.0000
29  Default   17.000000      household   17.000000   1.0000   1.0000
..      ...         ...            ...         ...      ...      ...
44  Topic10    0.684862        society   15.166894   0.7230  -4.1421
47  Topic10    0.685546       supports   15.201242   0.7218  -4.1411
9   Topic10    0.712624        collect   15.820509   0.7206  -4.1023
0   Topic10    0.690942   additionally   15.419719   0.7153  -4.1332
1   Topic10    0.690742        affairs   15.556257   0.7062  -4.1335
50  Topic10    0.708164           team   16.703374   0.6600  -4.1086
6   Topic10    0.707568      available   17.548962   0.6098  -4.1094
43  Topic10    0.686994         social   17.049943   0.6091  -4.1389
29  Topic10    0.689218      household   17.865405   0.5656  -4.1357
14  Topic10    0.701853      conducted   18.220043   0.5641  -4.1175
10  Topic10    0.719762      collected   20.397556   0.4764  -4.0923
53  Topic10    0.736644         united   21.542846   0.4450  -4.0692
52  Topic10    0.847754         unesco   41.991243  -0.0820  -3.9287
16  Topic10    0.845622       covering   42.448420  -0.0953  -3.9312
45  Topic10    0.955465  socioeconomic   79.897314  -0.6056  -3.8091
49  Topic10    0.950767    sustainable   79.889346  -0.6105  -3.81

In [44]:
pyLDAvis.sklearn.prepare(best_lda_model, Tfidf, vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.101977 -0.011765       1        1  43.378941
7      0.307621 -0.140392       2        1  31.857491
8      0.176252  0.260162       3        1   7.461834
9     -0.303328 -0.193251       4        1   3.499299
2     -0.105149  0.186289       5        1   2.845822
0      0.049814 -0.030932       6        1   2.191388
4     -0.006173 -0.016953       7        1   2.191307
1     -0.001824 -0.018199       8        1   2.191307
6     -0.009469 -0.014334       9        1   2.191305
3     -0.005765 -0.020626      10        1   2.191305, topic_info=   Category        Freq           Term       Total  loglift  logprob
24  Default  123.000000      education  123.000000  30.0000  30.0000
57  Default   60.000000          world   60.000000  29.0000  29.0000
38  Default   34.000000     population   34.000000  28.0000  28.0000
18  Default   43.000000        dataset   43.000000  27.0000  27.0000
45  Default   79.000000  socioeconomic   79.000000  26.0000  26.0000
49  Default   79.000000    sustainable   79.000000  25.0000  25.0000
17  Default   98.000000           data   98.000000  24.0000  24.0000
22  Default   74.000000    development   74.000000  23.0000  23.0000
30  Default  126.000000     indicators  126.000000  22.0000  22.0000
54  Default   94.000000          urban   94.000000  21.0000  21.0000
19  Default   72.000000    demographic   72.000000  20.0000  20.0000
25  Default   33.000000         global   33.000000  19.0000  19.0000
28  Default   35.000000         health   35.000000  18.0000  18.0000
46  Default   67.000000     statistics   67.000000  17.0000  17.0000
56  Default   42.000000        various   42.000000  16.0000  16.0000
52  Default   41.000000         unesco   41.000000  15.0000  15.0000
39  Default   42.000000         portal   42.000000  14.0000  14.0000
16  Default   42.000000       covering   42.000000  13.0000  13.0000
15  Default   47.000000       contains   47.000000  12.0000  12.0000
12  Default   21.000000       compiled   21.000000  11.0000  11.0000
32  Default   38.000000       national   38.000000  10.0000  10.0000
48  Default   26.000000        surveys   26.000000   9.0000   9.0000
10  Default   20.000000      collected   20.000000   8.0000   8.0000
7   Default   26.000000       censuses   26.000000   7.0000   7.0000
5   Default   26.000000    authorities   26.000000   6.0000   6.0000
51  Default   26.000000      transport   26.000000   5.0000   5.0000
35  Default   26.000000    observatory   26.000000   4.0000   4.0000
31  Default   28.000000          local   28.000000   3.0000   3.0000
23  Default   21.000000       economic   21.000000   2.0000   2.0000
29  Default   17.000000      household   17.000000   1.0000   1.0000
..      ...         ...            ...         ...      ...      ...
44  Topic10    0.684862        society   15.166894   0.7230  -4.1421
47  Topic10    0.685546       supports   15.201242   0.7218  -4.1411
9   Topic10    0.712624        collect   15.820509   0.7206  -4.1023
0   Topic10    0.690942   additionally   15.419719   0.7153  -4.1332
1   Topic10    0.690742        affairs   15.556257   0.7062  -4.1335
50  Topic10    0.708164           team   16.703374   0.6600  -4.1086
6   Topic10    0.707568      available   17.548962   0.6098  -4.1094
43  Topic10    0.686994         social   17.049943   0.6091  -4.1389
29  Topic10    0.689218      household   17.865405   0.5656  -4.1357
14  Topic10    0.701853      conducted   18.220043   0.5641  -4.1175
10  Topic10    0.719762      collected   20.397556   0.4764  -4.0923
53  Topic10    0.736644         united   21.542846   0.4450  -4.0692
52  Topic10    0.847754         unesco   41.991243  -0.0820  -3.9287
16  Topic10    0.845622       covering   42.448420  -0.0953  -3.9312
45  Topic10    0.955465  socioeconomic   79.897314  -0.6056  -3.8091
49  Topic10    0.950767    sustainable   79.889346  -0.6105  -3.81